### 1. mon_feature.pkl upload


In [1]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = './content/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'num_outgoing_packets',
    'avg_incoming_burst_size', 'cumul_max',
    'avg_incoming_order_first_30', 'outgoing_first_30'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)


       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
18995                    43.91               0.143962   
18996                    15.60               0.019465   
18997                    14.93               0.016411   
18998                    19.91               0.033281   
18999                    13.76               0.011074   

       avg_outgoing_burst_size  num_outgoing_packets  avg_incoming_burst_size  \
0                     1.551282                 121.0                16.666667   
1                     1.702128                  80.0                 9.319149   
2                     1.552632                 118.0                16.3

### 2. Random Forest


Split the dataset into training and testing sets


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Train and test Random Forest


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)

print("============ Before Hyperparameter tuning ============")
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_rf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


============ Before Hyperparameter tuning ============
RF Accuracy: 0.7970526315789473
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.69      0.76        51
           1       0.88      0.79      0.83        47
           2       0.76      0.92      0.83        48
           3       0.83      0.78      0.81        37
           4       0.79      0.86      0.83        44
           5       0.94      0.84      0.89        56
           6       0.92      0.93      0.92        58
           7       0.72      0.77      0.75        53
           8       0.67      0.63      0.65        49
           9       0.57      0.68      0.62        38
          10       0.86      0.76      0.81        55
          11       0.88      0.93      0.90        45
          12       0.91      0.91      0.91        55
          13       0.67      0.62      0.64        45
          14       0.76      0.71      0.73        52
          15       0.71 

### 3. Hyperparameter tuning by using Grid Search


In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10,20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

search = HalvingGridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1), 
                             param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train, y_train)
y_pred_tuned = search.predict(X_test)

print("============ After Hyperparameter tuning ============")
print("Best Parameters:", search.best_params_)
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))


============ After Hyperparameter tuning ============
Best Parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
RF Accuracy: 0.8195789473684211
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.71      0.73        51
           1       0.95      0.83      0.89        47
           2       0.83      0.92      0.87        48
           3       0.89      0.89      0.89        37
           4       0.90      0.86      0.88        44
           5       0.90      0.84      0.87        56
           6       0.90      0.98      0.94        58
           7       0.81      0.81      0.81        53
           8       0.76      0.69      0.72        49
           9       0.56      0.71      0.63        38
          10       0.91      0.76      0.83        55
          11       0.89      0.91      0.90        45
          12       0.91      0.93      0.92        55
    